The aim is toread 30 year daily rainfall data
1) plot annual mean rainfall difference between

   
   Model#1 G6sulfur-SSP245, G6solar-SSP245, solar-sulfur
   
   Model#2 G6sulfur-SSP245, G6solar-SSP245, solar-sulfur

In [ ]:
import numpy as np
import xarray as xr
# our local module:
import itcz

import matplotlib as mpl
import matplotlib.pyplot as plt

In [ ]:
import xarray as xr
from pathlib import Path
import myfunctions as mf

In [ ]:
# =========================
# User-defined metadata
# =========================

varname = "pr"

# =========================
# Base CEDA paths
# =========================

CEDA_BASE = Path("/badc/cmip6/data/CMIP6")

In [ ]:
#Model Names
MODELS = {
    "UKESM1-0-LL":  {"institution": "MOHC",         "ensemble": "r1i1p1f2",  "grid": "gn",},
    "CNRM-ESM2-1":  {"institution": "CNRM-CERFACS", "ensemble": "r1i1p1f2",  "grid": "gr",},
    "MPI-ESM1-2-LR":{"institution": "MPI-M",        "ensemble": "r1i1p1f1",  "grid": "gn",},
    "CESM2-WACCM":  {"institution": "NCAR",         "ensemble": "r1i1p1f1",  "grid": "gn",},
    "IPSL-CM6A-LR": {"institution": "IPSL",         "ensemble": "r1i1p1f1",  "grid": "gr",},
}


In [ ]:
#Experiment details
EXPERIMENTS = {
    # "HIST":     {"project": "CMIP",        "scenario": "historical", "color": "black"},
    "SSP245":   {"project": "ScenarioMIP", "scenario": "ssp245"},
    # "SSP585":   {"project": "ScenarioMIP", "scenario": "ssp585"},
    "G6solar":  {"project": "GeoMIP",      "scenario": "G6solar"},
    "G6sulfur": {"project": "GeoMIP",      "scenario": "G6sulfur"},
}

In [ ]:
# ==============================
# Annual-mean rainfall storage
# ==============================
pr_annmean_by_exp_2021_2050 = {}
pr_annmean_by_exp_2071_2100 = {}

for model_name, model_meta in MODELS.items():
    var = {}
    var_by_year = {}

    # --- LOAD DATA FOR A MODEL ---
    for exp, meta in EXPERIMENTS.items():

        # ensemble logic (unchanged)
        if model_name == "CESM2-WACCM":
            ensemble = "r1i1p1f2" if meta["scenario"] == "G6sulfur" else "r1i1p1f1"
        else:
            ensemble = model_meta["ensemble"]

        base = (
            CEDA_BASE
            / meta["project"]
            / model_meta["institution"]
            / model_name
            / meta["scenario"]
            / ensemble
            / "Amon"
            / varname
            / model_meta["grid"]
            / "latest"
        )
        print(base)

        if model_name == "CESM2-WACCM":
            if meta["scenario"] == "G6sulfur":
                ds = mf.open_files_CESM_G6sulfur(base)
            elif meta["scenario"] == "ssp585":
                ds = mf.open_files_CESM_ssp585(base)
            else:
                ds = mf.open_files(str(base))
        elif model_name == "IPSL-CM6A-LR":
            if meta["scenario"] == "ssp585":
                ds = mf.open_files_IPSL_ssp585(base)
            else:
                ds = mf.open_files(str(base))
        else:
            ds = mf.open_files(str(base))

        var[exp] = mf.read_var(ds, varname)

    # --- ANNUAL mean by year ---
    for exp, da in var.items():
        var_by_year[exp] = {
            "ANN": mf.seasonal_mean_by_year_old(da, 1, 12)
        }

    # --- init model keys ---
    pr_annmean_by_exp_2021_2050[model_name] = {}
    pr_annmean_by_exp_2071_2100[model_name] = {}

    for exp in ["SSP245", "G6sulfur", "G6solar"]:

        # 2021–2050
        pr_annmean_by_exp_2021_2050[model_name][exp] = (
            var_by_year[exp]["ANN"]
            .sel(year=slice(2021, 2050))
            .mean("year")
        )

        # 2071–2100
        pr_annmean_by_exp_2071_2100[model_name][exp] = (
            var_by_year[exp]["ANN"]
            .sel(year=slice(2071, 2100))
            .mean("year")
        )

In [ ]:
MODELS

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

# ==============================
# Plot settings
# ==============================
models = list(MODELS.keys())
n_models = len(models)

fig, axes = plt.subplots(
    nrows=n_models,
    ncols=3,
    figsize=(14, 2.0 * n_models),  # slightly shorter rows
    subplot_kw={"projection": ccrs.Robinson()}
)

fig.subplots_adjust(
    hspace=-0.10,   # vertical spacing between rows
    wspace=0.02,   # horizontal spacing between columns
    bottom=0.12    # room for colorbar
)

sec_per_day = 86400.0

# Ensure axes is 2D even if n_models == 1
if n_models == 1:
    axes = np.expand_dims(axes, axis=0)

vmin, vmax = -2, 2
cmap = "BrBG"

# ==============================
# Loop over models
# ==============================

lat_max = 50
lat_min = -lat_max

for i, model in enumerate(models):

    pr_ssp = (
        pr_annmean_by_exp_2021_2050[model]["SSP245"].sel(lat=slice(lat_min, lat_max))* sec_per_day)
    
    pr_sul = (
        pr_annmean_by_exp_2021_2050[model]["G6sulfur"].sel(lat=slice(lat_min, lat_max))* sec_per_day)
    
    pr_sol = (
        pr_annmean_by_exp_2021_2050[model]["G6solar"].sel(lat=slice(lat_min, lat_max))* sec_per_day)

    diffs = [
        pr_sul - pr_ssp,   # sulfur - SSP
        pr_sol - pr_ssp,   # solar - SSP
        pr_sol - pr_sul    # solar - sulfur
    ]

    titles = [
        "G6sulfur − SSP245",
        "G6solar − SSP245",
        "G6solar − G6sulfur"
    ]

    for j in range(3):
        ax = axes[i, j]
        ax.set_global()
        ax.coastlines(linewidth=0.3)
        # ax.add_feature(cfeature.BORDERS, linewidth=0.3)

        im = diffs[j].plot(
            ax=ax,
            transform=ccrs.PlateCarree(),
            cmap=cmap,
            vmin=vmin,
            vmax=vmax,
            add_colorbar=False
        )
        ax.set_extent([-180, 180, lat_min, lat_max], crs=ccrs.PlateCarree())

        if i == 0:
            ax.set_title(titles[j], fontsize=11)

        if j == 0:
            ax.text(
                -0.08, 0.5, model,
                transform=ax.transAxes,
                rotation=90,
                va="center",
                ha="right",
                fontsize=11
            )

# ==============================
# Shared colorbar
# ==============================
cbar = fig.colorbar(
    im,
    ax=axes,
    orientation="horizontal",
    fraction=0.05,
    pad=0.05
)

cbar.set_label("JJAS mean rainfall difference (mm day$^{-1}$): 2021-2050, immediately after SAI")


#Saving figure
fig_directory='/home/users/bidyut/20260112_Basic_Analysis'
figname="Figure_Rainfall_Response_2021-2050_Summer"
# plt.savefig(''+str(fig_directory)+'/'+str(figname)+'.svg', format="svg",transparent=True, dpi=1200)
# !rsvg-convert -f pdf -o {fig_directory}/{figname}.pdf {fig_directory}/{figname}.svg
plt.savefig(''+str(fig_directory)+'/'+str(figname)+'.pdf', format="pdf")

plt.show()

In [ ]:
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

# ==============================
# Plot settings
# ==============================
models = list(MODELS.keys())
n_models = len(models)

fig, axes = plt.subplots(
    nrows=n_models,
    ncols=3,
    figsize=(14, 2.0 * n_models),  # slightly shorter rows
    subplot_kw={"projection": ccrs.Robinson()}
)

fig.subplots_adjust(
    hspace=-0.10,   # vertical spacing between rows
    wspace=0.02,   # horizontal spacing between columns
    bottom=0.12    # room for colorbar
)

sec_per_day = 86400.0

# Ensure axes is 2D even if n_models == 1
if n_models == 1:
    axes = np.expand_dims(axes, axis=0)

vmin, vmax = -2, 2
cmap = "BrBG"

# ==============================
# Loop over models
# ==============================

lat_max = 50
lat_min = -lat_max

for i, model in enumerate(models):

    pr_ssp = (
        pr_annmean_by_exp_2071_2100[model]["SSP245"].sel(lat=slice(lat_min, lat_max))* sec_per_day)
    
    pr_sul = (
        pr_annmean_by_exp_2071_2100[model]["G6sulfur"].sel(lat=slice(lat_min, lat_max))* sec_per_day)
    
    pr_sol = (
        pr_annmean_by_exp_2071_2100[model]["G6solar"].sel(lat=slice(lat_min, lat_max))* sec_per_day)

    diffs = [
        pr_sul - pr_ssp,   # sulfur - SSP
        pr_sol - pr_ssp,   # solar - SSP
        pr_sol - pr_sul    # solar - sulfur
    ]

    titles = [
        "G6sulfur − SSP245",
        "G6solar − SSP245",
        "G6solar − G6sulfur"
    ]

    for j in range(3):
        ax = axes[i, j]
        ax.set_global()
        ax.coastlines(linewidth=0.3)
        # ax.add_feature(cfeature.BORDERS, linewidth=0.3)

        im = diffs[j].plot(
            ax=ax,
            transform=ccrs.PlateCarree(),
            cmap=cmap,
            vmin=vmin,
            vmax=vmax,
            add_colorbar=False
        )
        ax.set_extent([-180, 180, lat_min, lat_max], crs=ccrs.PlateCarree())

        if i == 0:
            ax.set_title(titles[j], fontsize=11)

        if j == 0:
            ax.text(
                -0.08, 0.5, model,
                transform=ax.transAxes,
                rotation=90,
                va="center",
                ha="right",
                fontsize=11
            )

# ==============================
# Shared colorbar
# ==============================
cbar = fig.colorbar(
    im,
    ax=axes,
    orientation="horizontal",
    fraction=0.05,
    pad=0.05
)

cbar.set_label("Summer mean rainfall difference (mm day$^{-1}$): 2071-2100, continued application of SAI")

#Saving figure
fig_directory='/home/users/bidyut/20260112_Basic_Analysis'
figname="Figure_Rainfall_Response_2071-2100_Summer"
# plt.savefig(''+str(fig_directory)+'/'+str(figname)+'.svg', format="svg",transparent=True, dpi=1200)
# !rsvg-convert -f pdf -o {fig_directory}/{figname}.pdf {fig_directory}/{figname}.svg
plt.savefig(''+str(fig_directory)+'/'+str(figname)+'.pdf', format="pdf")

plt.show()